In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib
import matplotlib.pyplot as plt

import os

In [2]:
path_to_datasets = '../Data/related_datasets'

# Import & Join datasets

## Sufficiencies

In [22]:
path_to_suff = os.path.join(path_to_datasets,'country_sufficiencies.csv')
suff = pd.read_csv(path_to_suff)

In [23]:
suff

,ISO3,2000,ssp1,ssp2,ssp3,ssp4,ssp5
0,AFG,0.712556,0.536646,0.468967,0.390449,0.367319,0.655464
1,AGO,0.591377,0.453087,0.571087,0.381656,0.558107,0.686487
2,ALB,2.329763,2.544620,2.428040,2.353029,2.508106,2.448250
3,ARE,0.417912,0.513526,0.515186,0.525990,0.529507,0.520904
4,ARG,7.931488,5.890471,6.701443,6.249030,6.205168,6.367101
5,ARM,1.878032,2.100658,2.219366,1.968406,2.177998,2.338979
6,AUS,10.710803,4.663941,4.728667,8.695204,5.828738,4.555344
7,AUT,2.429237,2.164068,2.273159,2.369145,2.180175,2.220470
8,AZE,2.136397,1.791686,1.848153,1.695168,1.818989,1.737322
9,BDI,0.458729,0.299079,0.299656,0.246161,0.248014,0.331084


## Import independency

In [8]:
path_to_gpck = os.path.join(path_to_datasets,'Import_independency/Matti_Kummu_2019/shp_trade_dep.gpkg')
import_indep = gpd.read_file(path_to_gpck)#, layer='countries')

In [10]:
import_indep[['admin','country','yr2013']]#,'geometry']]

,admin,country,yr2013,geometry
0,Saint Barthelemy,None,NaN,"(POLYGON ((-62.8319336 17.8764648, -62.8654297..."
1,Jersey,None,NaN,"(POLYGON ((-2.0186523 49.23125, -2.2358398 49...."
2,Ashmore and Cartier Islands,None,NaN,"(POLYGON ((123.5945312 -12.4256836, 123.595214..."
3,Siachen Glacier,None,NaN,"(POLYGON ((77.79941410000001 35.4958984, 77.04..."
4,Indian Ocean Territories,None,NaN,"(POLYGON ((105.7253906 -10.4929687, 105.696875..."
5,South Georgia and South Sandwich Islands,None,NaN,"(POLYGON ((-37.1033203 -54.065625, -36.3264648..."
6,Heard Island and McDonald Islands,None,NaN,"(POLYGON ((73.7074219 -53.1371094, 73.4651367 ..."
7,Guernsey,None,NaN,"(POLYGON ((-2.5123047 49.4945313, -2.5473633 4..."
8,Armenia,Armenia,0.124763,"(POLYGON ((46.490625 38.9066895, 46.1144531 38..."
9,Australia,Australia,1.000000,"(POLYGON ((145.0429688 -40.7867188, 146.317480..."


In [ ]:
## Join with Sufficiencies

sufficiencies = 

## Water Security 

* **Source**: Brauman, K. A., B. D. Richter, S. Postel, M. Malsy and M. Flörke (2016). "Water depletion: An improved metric for incorporating seasonal and dry-year water scarcity into water risk assessments." Elementa: Science of the Anthropocene 4(1): 000083

* **Aggregation by country**: aggregate it by country as the %area in any of the depleted category (i.e Categories: 5 to 8 )—> Looking at column OrigDepletionCategory


In [3]:
path_to_tif = os.path.join(path_to_datasets,'Water_Security/GlobalWaterScarcity/DepletionMapCurrent.tif')

In [5]:
# tif to tabular

In [ ]:
# aggregate

## Food Security
* Dataset: GFSI -- Global Food Security Index 2018, for 113 countries
* Source: EIU Global Food Security Index 2018 Building resilience in the face of rising food-security risks, 2018



In [19]:
path_to_file = os.path.join(path_to_datasets,'Food_Security/GFSI_2018_min.csv')
food_secu = pd.read_excel(path_to_file)

In [21]:
food_secu

,Country_name,GFSI_2018
0,Singapore,85.9
1,Ireland,85.5
2,United Kingdom,85.0
3,United States,85.0
4,Netherlands,84.7
5,Australia,83.7
6,Switzerland,83.5
7,Finland,83.3
8,Canada,83.2
9,France,82.9


In [ ]:
## Join with Sufficiencies

sufficiencies = 

## Malnutrition
To download from https://data.worldbank.org/indicator/SH.STA.MALN.ZS


# Scatterplots